In [1]:
import sys
import os

sys.path.append(os.path.abspath("../"))

from src.data.data_loading import load_cleaned_dataset

# Load the cleaned dataset
df = load_cleaned_dataset()

# Display the first few rows
print(df.head())

      bearer_id                start  start_ms                  end  end_ms  \
0  1.311448e+19  2019-04-04 12:01:00     770.0  2019-04-25 14:35:00   662.0   
1  1.311448e+19  2019-04-09 13:04:00     235.0  2019-04-25 08:15:00   606.0   
2  1.311448e+19  2019-04-09 17:42:00      10.0  2019-04-25 11:58:00   652.0   
3  1.311448e+19  2019-04-10 00:31:00     486.0  2019-04-25 07:36:00   171.0   
4  1.311448e+19  2019-04-12 20:10:00     565.0  2019-04-25 10:40:00   954.0   

   dur._(ms)          imsi  msisdn/number          imei  \
0   398786.0  2.082014e+14   3.366496e+10  3.552121e+13   
1   398786.0  2.082019e+14   3.368185e+10  3.579401e+13   
2   398786.0  2.082003e+14   3.376063e+10  3.528151e+13   
3   398786.0  2.082014e+14   3.375034e+10  3.535661e+13   
4   398786.0  2.082014e+14   3.369980e+10  3.540701e+13   

      last_location_name  ...  netflix_dl_(bytes)  netflix_ul_(bytes)  \
0  9.16456699548519E+015  ...           8198936.0           9656251.0   
1                L77566A